## Initialization for Training and Analysis

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

import os, re, json, time, copy, csv, glob, itertools, pickle, warnings, shutil

import numpy as np
from numpy import genfromtxt

import pandas as pd
from pandas.tools.plotting import table
pd.options.display.max_columns = 100

import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore', category=FutureWarning)
from pathlib import Path

import helper as ksn

In [ ]:
datasets = 'D1-IsolatedNetworkPassiveDFP'
datasets = 'D21-CampusNetworkActiveDFP'
datasets = 'D22-CampusNetworkPassiveDFP'

In [2]:
home = str(Path.home())
#data_folder = 'datasets'
data_folder = os.path.join(home, 'N', 'fingerprinting', datasets)
out_folder = 'r_' + datasets

sample = 'yes'
sample_size = 5
batch = 128
epochs = 20

f1 = os.path.join(out_folder, 'results_training.pkl')
f2 = os.path.join(out_folder, 'results_testing.pkl')
f3 = os.path.join(out_folder, 'data.pkl')

if sample == 'yes':
    epochs = 2
data_dirs = [os.path.join(data_folder, i) for i in sorted(os.listdir(data_folder)) if not i.startswith('.')]
Loss, Accuracy, Time = 0, 1, 2

## Training Model

In [3]:
data = dict()
for data_dir in data_dirs:
    n_files = []
    n_files.append(len(os.listdir(os.path.join(data_dir, 'training'))))
    n_files.append(len(glob.glob(os.path.join(data_dir, 'training' , '*', '*.png'))))
    n_files.append(len(os.listdir(os.path.join(data_dir, 'testing'))))
    n_files.append(len(glob.glob(os.path.join(data_dir, 'testing' , '*', '*.png'))))
    data[data_dir] = n_files
print('Total Datasets are {}'.format(len(data_dirs)))
#print(data)

Total Datasets are 30


In [4]:
#for dr in data_dirs:
#    if os.path.exists(os.path.join(dr, 'results.csv')):
#        cmd = "rm " + os.path.join(dr, 'results.csv')
#        os.system(cmd)

if os.path.exists(out_folder):
    print(out_folder + ' already exists')
    assert False
    #os.system("rm -r results")
    os.system("mkdir " + out_folder)
else:
    os.system("mkdir " + out_folder)

In [5]:
results_training = dict()
results_testing = dict()
for dr in data_dirs:
    classes = data[dr][0]
    print(), print('Processing {} with classes {}'.format(dr, classes))

    image_datasets, dataloaders = \
        ksn.get_dataloader(os.path.join(dr, 'training'), os.path.join(dr, 'testing'), batch)
    
    training_loss, testing_loss, training_accuracy, \
    testing_accuracy, training_time, testing_time, \
    all_labels_flat, all_preds_flat = \
        ksn.train_model(image_datasets, dataloaders, dr, out_folder, classes, sample=sample, epochs=epochs)
    
    results_training[dr] = [training_loss, training_accuracy, training_time]
    results_testing[dr] = [testing_loss, testing_accuracy, testing_time]
    
    with open(f1, 'wb') as f:
        pickle.dump(results_training, f, pickle.HIGHEST_PROTOCOL)

    with open(f2, 'wb') as f:
        pickle.dump(results_testing, f, pickle.HIGHEST_PROTOCOL)

    with open(f3, 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)


Processing /home/ksn/N/fingerprinting/datasets/Iperf_Tcp_Case1_Passive with classes 7
Best Testing Accuracy 0.006

Processing /home/ksn/N/fingerprinting/datasets/Iperf_Tcp_Case1_Passive_ID with classes 36
Best Testing Accuracy 0.001

Processing /home/ksn/N/fingerprinting/datasets/Iperf_Tcp_Case2_Isolated with classes 5
Best Testing Accuracy 0.015

Processing /home/ksn/N/fingerprinting/datasets/Iperf_Tcp_Case2_Isolated_ID with classes 14
Best Testing Accuracy 0.007

Processing /home/ksn/N/fingerprinting/datasets/Iperf_Tcp_Case2_Passive with classes 5
Best Testing Accuracy 0.043

Processing /home/ksn/N/fingerprinting/datasets/Iperf_Tcp_Case2_Passive_ID with classes 14
Best Testing Accuracy 0.022

Processing /home/ksn/N/fingerprinting/datasets/Iperf_Udp_Case1_Isolated with classes 5
Best Testing Accuracy 0.265

Processing /home/ksn/N/fingerprinting/datasets/Iperf_Udp_Case1_Isolated_ID with classes 14
Best Testing Accuracy 0.060

Processing /home/ksn/N/fingerprinting/datasets/Iperf_Udp_Ca

In [ ]:
with open(f1, 'wb') as f:
    pickle.dump(results_training, f, pickle.HIGHEST_PROTOCOL)

with open(f2, 'wb') as f:
    pickle.dump(results_testing, f, pickle.HIGHEST_PROTOCOL)

with open(f3, 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
#print(results_training)
#print()
#print(results_testing)